Import relevant libraries

In [31]:
import pandas as pd
import numpy as np

Read the cartel file

In [32]:
# read cartel excel file
cartel_df = pd.read_excel('./transformed_data/base/cartels/cartels.xlsx')

# count number of rows in cartel_df
print(f"number of records in cartel_df:  {cartel_df.shape[0]}")

cartel_df.head(10)

number of records in cartel_df:  292


,case,cartel,entity_name,key_iustin,key_invented,start,end,duration,number_of_firms,number_cartel_firms,sector,subsector,cartel_classification
0,31865,PVC (II),Elf Aquitaine SA,C000007728,31865C000007728,1981,1994,14,6,12.0,3,3,"1.2, 1.6"
1,31865,PVC (II),BASF SE,C000008351,31865C000008351,1981,1994,14,6,12.0,3,3,"1.2, 1.6"
2,31865,PVC (II),Koninklijke DSM,C000013105,31865C000013105,1981,1994,14,6,12.0,3,3,"1.2, 1.6"
3,31865,PVC (II),ENI,C901505544,31865C901505544,1981,1994,14,6,12.0,3,3,"1.2, 1.6"
4,31865,PVC (II),Wacker Chemie AG,C901711026,31865C901711026,1981,1994,14,6,12.0,3,3,"1.2, 1.6"
5,31865,PVC (II),Royal Dutch Shell,C901842957,31865C901842957,1981,1994,14,6,12.0,3,3,"1.2, 1.6"
6,31906,FLAT GLASS,PPG Industries Inc,C000003476,31906C000003476,1982,1987,6,2,3.0,3,4,"1.2, 1.6, 1.7"
7,31906,FLAT GLASS,Saint Gobain,C000007729,31906C000007729,1982,1987,6,2,3.0,3,4,"1.2, 1.6, 1.7"
8,32800,Quantel International - continuum / Quantel SA,Hoya Corporation,C000087990,32800C000087990,1986,1992,7,1,2.0,3,5,1.6
9,33016,ANSAC,FMC Corp.,C000001711,33016C000001711,1984,1990,7,4,7.0,3,3,1.5


Read the mapping file to map entity_name to file_names

In [33]:
# Get all filenames from mapping file (contains all filenames)
df_mapping = pd.read_excel("./raw_data/mapping/filename_to_entity_mapping.xlsx")

# if entity_name_direct_match column is not empty take entity_name_direct_match else take entity_name_indirect_match_1
df_mapping['entity_name'] = df_mapping.apply(
    lambda x: x['entity_name_direct_match'] if pd.notna(x['entity_name_direct_match']) else x['entity_name_indirect_match_1'], axis=1)

# drop all columns except entity_name and filename
df_mapping = df_mapping[['entity_name', 'file_name']]

# join cartel_df with df_mapping on entity_name
cartel_df = cartel_df.merge(df_mapping, on='entity_name', how='left')

# count number of rows in cartel_df
print(f"number of records in cartel_df:  {cartel_df.shape[0]}")

cartel_df.head(10)

number of records in cartel_df:  292


,case,cartel,entity_name,key_iustin,key_invented,start,end,duration,number_of_firms,number_cartel_firms,sector,subsector,cartel_classification,file_name
0,31865,PVC (II),Elf Aquitaine SA,C000007728,31865C000007728,1981,1994,14,6,12.0,3,3,"1.2, 1.6",NaN
1,31865,PVC (II),BASF SE,C000008351,31865C000008351,1981,1994,14,6,12.0,3,3,"1.2, 1.6",NaN
2,31865,PVC (II),Koninklijke DSM,C000013105,31865C000013105,1981,1994,14,6,12.0,3,3,"1.2, 1.6",NaN
3,31865,PVC (II),ENI,C901505544,31865C901505544,1981,1994,14,6,12.0,3,3,"1.2, 1.6",ENI
4,31865,PVC (II),Wacker Chemie AG,C901711026,31865C901711026,1981,1994,14,6,12.0,3,3,"1.2, 1.6",NaN
5,31865,PVC (II),Royal Dutch Shell,C901842957,31865C901842957,1981,1994,14,6,12.0,3,3,"1.2, 1.6",NaN
6,31906,FLAT GLASS,PPG Industries Inc,C000003476,31906C000003476,1982,1987,6,2,3.0,3,4,"1.2, 1.6, 1.7",NaN
7,31906,FLAT GLASS,Saint Gobain,C000007729,31906C000007729,1982,1987,6,2,3.0,3,4,"1.2, 1.6, 1.7",NaN
8,32800,Quantel International - continuum / Quantel SA,Hoya Corporation,C000087990,32800C000087990,1986,1992,7,1,2.0,3,5,1.6,NaN
9,33016,ANSAC,FMC Corp.,C000001711,33016C000001711,1984,1990,7,4,7.0,3,3,1.5,NaN


Get the amount of file_names per cartel (Count not null per cartel)

In [34]:
# Assuming your DataFrame is named df
# Step 1: Filter non-null file names
cartel_df_not_null = cartel_df[cartel_df['file_name'].notnull()]

# Step 2: Group by 'case' and 'cartel', and count non-null file_name entries
cartel_df_fn_counts = cartel_df_not_null.groupby(['case', 'cartel'])['file_name'].count().reset_index()

# Optional: Rename the column to something meaningful
cartel_df_fn_counts = cartel_df_fn_counts.rename(columns={'file_name': 'number_of_firms_with_file'})

cartel_df_fn_counts.head(10)

,case,cartel,number_of_firms_with_file
0,31865,PVC (II),1
1,37444,SAS Maersk Air and Sun-Air,1
2,37533,Choline Chloride,1
3,37773,MCAA (Monochloroacetic acid),1
4,37919,Bank charges for exchanging EUR currencies Ger...,1
5,38069,Copper plumbing tubes,3
6,38121,Fittings,3
7,38344,Prestressing Steel Producers,1
8,38432,Professional videotape,3
9,38443,Rubber chemicals,1


Join the counts of the previous step with the cartel file to get the number_of_firms_with_file for each cartel/Case

In [35]:
cartel_df = cartel_df.merge(cartel_df_fn_counts, left_on=['case', 'cartel'], right_on=['case', 'cartel'], how='left')

# fill NaN values with 0
cartel_df['number_of_firms_with_file'] = cartel_df['number_of_firms_with_file'].fillna(0)
# convert to int
cartel_df['number_of_firms_with_file'] = cartel_df['number_of_firms_with_file'].astype(int)

# fill NaN values with 0
cartel_df['number_cartel_firms'] = cartel_df['number_cartel_firms'].fillna(0)
# convert to int
cartel_df['number_cartel_firms'] = cartel_df['number_cartel_firms'].astype(int)

# fill NaN values with 0
cartel_df['number_of_firms'] = cartel_df['number_of_firms'].fillna(0)
# convert to int
cartel_df['number_of_firms'] = cartel_df['number_of_firms'].astype(int)

# change order of columns
cartel_df = cartel_df[['case', 'cartel', 'number_of_firms', 'number_cartel_firms', 'number_of_firms_with_file', 'sector', 'subsector', 'entity_name', 'file_name', 'duration', 'start', 'end', ]]

print(f"number of records in cartel_df_v2:  {cartel_df.shape[0]}")

print(cartel_df.dtypes)

cartel_df.head(10)


number of records in cartel_df_v2:  292
case                         object
cartel                       object
number_of_firms               int64
number_cartel_firms           int64
number_of_firms_with_file     int64
sector                        int64
subsector                     int64
entity_name                  object
file_name                    object
duration                      int64
start                         int64
end                           int64
dtype: object


,case,cartel,number_of_firms,number_cartel_firms,number_of_firms_with_file,sector,subsector,entity_name,file_name,duration,start,end
0,31865,PVC (II),6,12,1,3,3,Elf Aquitaine SA,NaN,14,1981,1994
1,31865,PVC (II),6,12,1,3,3,BASF SE,NaN,14,1981,1994
2,31865,PVC (II),6,12,1,3,3,Koninklijke DSM,NaN,14,1981,1994
3,31865,PVC (II),6,12,1,3,3,ENI,ENI,14,1981,1994
4,31865,PVC (II),6,12,1,3,3,Wacker Chemie AG,NaN,14,1981,1994
5,31865,PVC (II),6,12,1,3,3,Royal Dutch Shell,NaN,14,1981,1994
6,31906,FLAT GLASS,2,3,0,3,4,PPG Industries Inc,NaN,6,1982,1987
7,31906,FLAT GLASS,2,3,0,3,4,Saint Gobain,NaN,6,1982,1987
8,32800,Quantel International - continuum / Quantel SA,1,2,0,3,5,Hoya Corporation,NaN,7,1986,1992
9,33016,ANSAC,4,7,0,3,3,FMC Corp.,NaN,7,1984,1990


Prepare the t_A... columns in the cartel file

In [36]:
# get largest duration value
max_duration = cartel_df['duration'].max()
print(f"max duration:   {max_duration}")

# get records with max duration
print(f"cartel name:    {cartel_df[cartel_df['duration'] == max_duration]["cartel"].unique()}")

cartel_df['t_A_-3'] = cartel_df['start'] - 3
cartel_df['t_A_-2'] = cartel_df['start'] - 2
cartel_df['t_A_-1'] = cartel_df['start'] - 1
cartel_df['t_A'] = cartel_df['start']

# for range 1 to max_duration create new column with name t_A_+1, t_A_+2, t_A_+3 and so on
for i in range(1, max_duration - 1):
# Use np.where to conditionally assign values
    cartel_df[f't_A_+{i}'] = np.where(cartel_df['start'] + i < cartel_df['end'], 
                                         cartel_df['start'] + i, 
                                         None)

cartel_df['t_E'] = cartel_df['end']
cartel_df['t_E_+1'] = cartel_df['end'] + 1
cartel_df['t_E_+2'] = cartel_df['end'] + 2
cartel_df['t_E_+3'] = cartel_df['end'] + 3

cartel_df.head(10)

max duration:   35
cartel name:    ['Feed Phosphates Producers']


,case,cartel,number_of_firms,number_cartel_firms,number_of_firms_with_file,sector,subsector,entity_name,file_name,duration,...,t_A_+28,t_A_+29,t_A_+30,t_A_+31,t_A_+32,t_A_+33,t_E,t_E_+1,t_E_+2,t_E_+3
0,31865,PVC (II),6,12,1,3,3,Elf Aquitaine SA,NaN,14,...,None,None,None,None,None,None,1994,1995,1996,1997
1,31865,PVC (II),6,12,1,3,3,BASF SE,NaN,14,...,None,None,None,None,None,None,1994,1995,1996,1997
2,31865,PVC (II),6,12,1,3,3,Koninklijke DSM,NaN,14,...,None,None,None,None,None,None,1994,1995,1996,1997
3,31865,PVC (II),6,12,1,3,3,ENI,ENI,14,...,None,None,None,None,None,None,1994,1995,1996,1997
4,31865,PVC (II),6,12,1,3,3,Wacker Chemie AG,NaN,14,...,None,None,None,None,None,None,1994,1995,1996,1997
5,31865,PVC (II),6,12,1,3,3,Royal Dutch Shell,NaN,14,...,None,None,None,None,None,None,1994,1995,1996,1997
6,31906,FLAT GLASS,2,3,0,3,4,PPG Industries Inc,NaN,6,...,None,None,None,None,None,None,1987,1988,1989,1990
7,31906,FLAT GLASS,2,3,0,3,4,Saint Gobain,NaN,6,...,None,None,None,None,None,None,1987,1988,1989,1990
8,32800,Quantel International - continuum / Quantel SA,1,2,0,3,5,Hoya Corporation,NaN,7,...,None,None,None,None,None,None,1992,1993,1994,1995
9,33016,ANSAC,4,7,0,3,3,FMC Corp.,NaN,7,...,None,None,None,None,None,None,1990,1991,1992,1993


Prepare the v_A... fields for the corresponding values to each time slot

In [37]:
cartel_df['v_A_-3'] = None
cartel_df['v_A_-2'] = None
cartel_df['v_A_-1'] = None
cartel_df['v_A'] = None

# for range 1 to max_duration create new column with name t_A_+1, t_A_+2, t_A_+3 and so on
for i in range(1, max_duration - 1):
# Use np.where to conditionally assign values
    cartel_df[f'v_A_+{i}'] = None

cartel_df['v_E'] = None
cartel_df['v_E_+1'] = None
cartel_df['v_E_+2'] = None
cartel_df['v_E_+3'] = None

Fill in the v_A... fields with the herfindal index

In [38]:
# crate copy of cartel_df
cartel_df_herfindal = cartel_df.copy()

# get all column names that start with t_A OR t_E
t_a_columns = [col for col in cartel_df_herfindal.columns if col.startswith('t_A') or col.startswith('t_E')]
# get all column names that start with v_A OR v_E
v_e_columns = [col for col in cartel_df_herfindal.columns if col.startswith('v_A') or col.startswith('v_E')]

print(t_a_columns)
print(v_e_columns)

def get_herfindahl_value(year, company_name):
    filtered = herfindal_df.loc[(herfindal_df['year'] == year) & (herfindal_df["company_name"] == company_name), 'herfindahl_index']
    return filtered.values[0] if not filtered.empty else np.nan

herfindal_df = pd.read_excel('./transformed_data/shareholder_data/company_metrics.xlsx')

for index, row in cartel_df_herfindal.iterrows():

    file_name = row['file_name']

    for col in t_a_columns:
        # get the value of the column
        t_a_value = row[col]
        # check if the value is not null
        if pd.notna(t_a_value):
            # get the herfindahl value for the year and company name
            herfindahl_value = get_herfindahl_value(t_a_value, file_name)
            # set the value in the corresponding v_A column
            cartel_df_herfindal.at[index, col.replace('t_', 'v_')] = herfindahl_value

print('Finished')

['t_A_-3', 't_A_-2', 't_A_-1', 't_A', 't_A_+1', 't_A_+2', 't_A_+3', 't_A_+4', 't_A_+5', 't_A_+6', 't_A_+7', 't_A_+8', 't_A_+9', 't_A_+10', 't_A_+11', 't_A_+12', 't_A_+13', 't_A_+14', 't_A_+15', 't_A_+16', 't_A_+17', 't_A_+18', 't_A_+19', 't_A_+20', 't_A_+21', 't_A_+22', 't_A_+23', 't_A_+24', 't_A_+25', 't_A_+26', 't_A_+27', 't_A_+28', 't_A_+29', 't_A_+30', 't_A_+31', 't_A_+32', 't_A_+33', 't_E', 't_E_+1', 't_E_+2', 't_E_+3']
['v_A_-3', 'v_A_-2', 'v_A_-1', 'v_A', 'v_A_+1', 'v_A_+2', 'v_A_+3', 'v_A_+4', 'v_A_+5', 'v_A_+6', 'v_A_+7', 'v_A_+8', 'v_A_+9', 'v_A_+10', 'v_A_+11', 'v_A_+12', 'v_A_+13', 'v_A_+14', 'v_A_+15', 'v_A_+16', 'v_A_+17', 'v_A_+18', 'v_A_+19', 'v_A_+20', 'v_A_+21', 'v_A_+22', 'v_A_+23', 'v_A_+24', 'v_A_+25', 'v_A_+26', 'v_A_+27', 'v_A_+28', 'v_A_+29', 'v_A_+30', 'v_A_+31', 'v_A_+32', 'v_A_+33', 'v_E', 'v_E_+1', 'v_E_+2', 'v_E_+3']
Finished


Save the excel for analysis

In [39]:
# save cartel_df_v2 to excel file
cartel_df.to_excel('./transformed_data/cartel_networks/test222.xlsx', index=False)

Calculate Ownership of investors on cartels by year

In [46]:
merged_shareholders_df = pd.read_excel('./transformed_data/connected_shareholder_data/merged_shareholder_data.xlsx')

# get all column names that start with perc_os
perc_os_columns = [col for col in merged_shareholders_df.columns if col.startswith('perc_os')]

# select only relevant columns
merged_shareholders_df = merged_shareholders_df[['investor_name', 'company_name'] + perc_os_columns]

merged_shareholders_df.head(10)

,investor_name,company_name,perc_os_31-Dec-2011,perc_os_31-Dec-2010,perc_os_31-Dec-2009,perc_os_31-Dec-2008,perc_os_31-Dec-2007,perc_os_31-Dec-2006,perc_os_31-Dec-2005,perc_os_31-Dec-2004,perc_os_31-Dec-2003,perc_os_31-Dec-2002,perc_os_31-Dec-2001,perc_os_31-Dec-2000,perc_os_31-Dec-1999,perc_os_31-Dec-1998,perc_os_31-Dec-1997
0,Étoile Gestion,Aalberts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WestLB Mellon Asset Management Kapitalanlagege...,Aalberts,0.03,0.05,0.03,0.14,0.17,0.05,0.06,0.07,NaN,NaN,0.0,0.12,NaN,0.0,0.11
2,Wealth Management Partners NV,Aalberts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.89,NaN,NaN,NaN
3,Waddell & Reed Investment Management Company,Aalberts,NaN,NaN,NaN,0.00,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,W & W Asset Management GmbH,Aalberts,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Vontobel Asset Management, Inc.",Aalberts,NaN,NaN,NaN,0.00,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Vanguard Group, Inc.",Aalberts,0.41,0.25,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,VPB Finance S.A.,Aalberts,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Universal-Investment-Gesellschaft mbH,Aalberts,0.01,NaN,0.00,0.34,0.29,0.57,0.63,0.15,0.0,0.09,NaN,0.00,0.06,NaN,NaN
9,Union Investment Group,Aalberts,0.00,0.04,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# join cartel_df with merged_shareholders_df on file_name = company_name
cartel_df_sh = cartel_df.merge(merged_shareholders_df, left_on='file_name', right_on='company_name', how='left')
# drop company_name column 
cartel_df_sh = cartel_df_sh.drop(columns=['company_name'])

# count rows in cartel_df_sh
print(f"number of records in cartel_df_sh:  {cartel_df_sh.shape[0]}")

t_columns = [col for col in merged_shareholders_df.columns if col.startswith('t_A') or col.startswith('t_E')]

for index, row in cartel_df_sh.iterrows():
    
    for col in t_columns:
        # get the value of the column
        t_value = row[col]
        # check if the value is not null
        if pd.notna(t_value):
            # get the perc_os value for the year and company name
            perc_os_value = row['perc_os_31-Dec-' + t_value]
            # set the value in the corresponding v_A column
            cartel_df_sh.at[index, col.replace('t_', 'v_')] = perc_os_value

            print(perc_os_value)

            break
    

number of records in cartel_df_sh:  69939


In [41]:
cartel_df.head(10)

,case,cartel,number_of_firms,number_cartel_firms,number_of_firms_with_file,sector,subsector,entity_name,file_name,duration,...,v_A_+28,v_A_+29,v_A_+30,v_A_+31,v_A_+32,v_A_+33,v_E,v_E_+1,v_E_+2,v_E_+3
0,31865,PVC (II),6,12,1,3,3,Elf Aquitaine SA,NaN,14,...,None,None,None,None,None,None,None,None,None,None
1,31865,PVC (II),6,12,1,3,3,BASF SE,NaN,14,...,None,None,None,None,None,None,None,None,None,None
2,31865,PVC (II),6,12,1,3,3,Koninklijke DSM,NaN,14,...,None,None,None,None,None,None,None,None,None,None
3,31865,PVC (II),6,12,1,3,3,ENI,ENI,14,...,None,None,None,None,None,None,None,None,None,None
4,31865,PVC (II),6,12,1,3,3,Wacker Chemie AG,NaN,14,...,None,None,None,None,None,None,None,None,None,None
5,31865,PVC (II),6,12,1,3,3,Royal Dutch Shell,NaN,14,...,None,None,None,None,None,None,None,None,None,None
6,31906,FLAT GLASS,2,3,0,3,4,PPG Industries Inc,NaN,6,...,None,None,None,None,None,None,None,None,None,None
7,31906,FLAT GLASS,2,3,0,3,4,Saint Gobain,NaN,6,...,None,None,None,None,None,None,None,None,None,None
8,32800,Quantel International - continuum / Quantel SA,1,2,0,3,5,Hoya Corporation,NaN,7,...,None,None,None,None,None,None,None,None,None,None
9,33016,ANSAC,4,7,0,3,3,FMC Corp.,NaN,7,...,None,None,None,None,None,None,None,None,None,None
